In [1]:
%%capture
!pip -q install openai
!pip -q install langchain-openai
!pip -q install langchain-core
!pip -q install langchain-community
!pip -q install sentence-transformers
!pip -q install langchain-huggingface
!pip -q install langchain-chroma
!pip -q install chromadb
!pip -q install pypdf
!pip -q install langchain
!pip -q install gradio


In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
import os
import gradio as gr
import openai

In [3]:
# Read OpenAI key from Colab Secrets

from google.colab import userdata

api_key = userdata.get('OPENAI_KEY')           # <-- change this as per your secret's name
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [4]:
d1 = '/content/vtransformers_d1.pdf'
d2 = '/content/rag_d2.pdf'
d3 = '/content/transformers_d3.pdf'
d4 = '/content/ai_d4.pdf'

# Define user roles and document access permissions
roles = {
    "admin": ["d1", "d2", "d3", "d4"],
    "contributor": ["d1", "d2", "d3"],
    "restricted-user": ["d3"],
}

# Define users and their roles
users = {
    "abc": {"role": "admin", "password": "abc"},
    "pqr": {"role": "contributor", "password": "pqr"},
    "rst": {"role": "restricted-user", "password": "rst"},
}


In [5]:
from langchain_community.document_loaders import PyPDFLoader

# Load documents
documents = {}
documents["d1"] = d1
documents["d2"] = d2
documents["d3"] = d3
documents["d4"] = d4

# Create a single vector store for all documents
all_docs = []
for doc_id in documents:
  all_docs.extend(PyPDFLoader(documents[doc_id]).load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_texts = text_splitter.split_documents(all_docs)
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(all_texts, embeddings)

In [6]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    k=3,
    output_key='answer'
)

<ipython-input-6-5267b4b43d68>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [20]:
def get_qa_chain(vectordb, accessible_doc_ids):
  accessible_docs = []
  for doc_id in accessible_doc_ids:
    accessible_docs.append(documents[doc_id])

  retriever = vectordb.as_retriever(search_type="mmr",
      search_kwargs={"k": 2, "fetch_k":5, "filter": {"source": {"$in": accessible_docs}}}
  )
  qa_chain = ConversationalRetrievalChain.from_llm(
        llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0),
        retriever=retriever,
        memory=memory,
        return_source_documents=True
    )
  print(f"qa chain: {qa_chain}")
  return qa_chain


In [21]:
def process_query(username, password, query):
    if username not in users or users[username]["password"] != password:
        return "Incorrect username or password.", "", ""
    user_role = users[username]["role"]
    accessible_doc_ids = roles[user_role]

    if not accessible_doc_ids:
        return "No documents available for your role.", "", ""

    qa_chain = get_qa_chain(vectordb, accessible_doc_ids)
    result = qa_chain({"question": query})

    # Get metadata of retrieved documents
    retrieved_docs_metadata = ""
    if 'source_documents' in result:
        for doc in result['source_documents']:
            if hasattr(doc, 'metadata'):
                retrieved_docs_metadata += f"Metadata: {doc.metadata}\n"

    return (result["answer"], user_role, retrieved_docs_metadata)

## User Interface: Gradio

In [22]:
demo = gr.Interface(
    fn=process_query,
    inputs=[
        gr.Textbox(label="Username"),
        gr.Textbox(label="Password", type="password"),
        gr.Textbox(label="Query"),
    ],
    outputs=[
        gr.Textbox(label="Response"),
        gr.Textbox(label="User Role"),
        gr.Textbox(label="Document Metadata")
    ],
    title="RAG System with User Access Control",
)

demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://eaa020de1d602d5e74.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


qa chain: memory=ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Who is the Prime Minister of India', additional_kwargs={}, response_metadata={}), AIMessage(content="I don't know.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain transformers', additional_kwargs={}, response_metadata={}), AIMessage(content='Transformers are a type of neural network architecture that was originally designed for natural language processing (NLP) tasks. They utilize mechanisms such as self-attention to process and generate sequences of data, allowing them to capture long-range dependencies and relationships within the data effectively. Transformers have gained significant attention due to their ability to achieve state-of-the-art performance in various tasks, not only in NLP but also in other fields such as image generation, music composition, and scientific research. Their versatility and robust frameworks have made them a s